In [1]:
from cross import cross_hyperbolic_11_2p1o, cross_hyperbolic_11_2p2o
from implicit import implicit_hyperbolic_11_2p1o, implicit_hyperbolic_11_2p2o

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import math

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [3]:
def select_solver(scheme, approximation):
    val = scheme * 10 + approximation
    solver = None
    if val == 0:
        solver = cross_hyperbolic_11_2p1o
    elif val == 1:
        solver = cross_hyperbolic_11_2p2o
    elif val == 10:
        solver = implicit_hyperbolic_11_2p1o
    elif val == 11:
        solver = implicit_hyperbolic_11_2p2o
    else:
        raise ValueError

    return scheme, solver

In [4]:
t_begin_widget = widgets.Text('0.0')
t_end_widget = widgets.Text('1.0')
current_t_widget = widgets.FloatSlider(min=0.0, max=1.0, step=0.01)

def update_current_t_range(*args):
    try:
        begin = float(t_begin_widget.value)
        end = float(t_end_widget.value)
        current_t_widget.min = begin
        current_t_widget.max = end
        current_t_widget.step = (end - begin) / 100
    except Exception:
        print('error')
        
t_begin_widget.observe(update_current_t_range, 'value')
t_end_widget.observe(update_current_t_range, 'value')

def solve(scheme, approximation, a, b, c, q, x_begin, x_end, t_begin, t_end, h=0.01, tau=0.005, current_t=0.0):
    scheme, solver = select_solver(scheme, approximation) 
    try:
        a = float(a) 
        b = float(b)
        c = float(c)
        q = float(q)

        x_begin = float(x_begin)
        x_end = float(x_end)

        t_begin = float(t_begin)
        t_end = float(t_end)
        
        h = float(h)
        tau = float(tau)
    except Exception:
        print('Please, recheck your input')
        return

    def phi1(t):
        return 0

    def phi2(t):
        return 0

    def psi1(x):
        return 0

    def psi2(x):
        return 2 * np.exp(-x) * np.sin(x)

    def f(x, t):
        return 0

    def u_ref(x, t):
        return np.exp(-t - x) * np.sin(x) * np.sin(2 * t)

    nx = round((x_end - x_begin) / h) + 1
    nt = round((t_end - t_begin) / tau) + 1

    x = np.linspace(x_begin, x_end, nx)
    
    u = solver(a, b, c, q, x_begin, x_end, t_begin, t_end, h, tau, phi1, phi2, psi1, psi2, f)
    n = math.floor((current_t - t_begin) / tau)
    
    u_ref_vals = u_ref(x, t_begin + tau * n)
    
    fig, ax = plt.subplots()
    ax.plot(x, u[n], 'g', label='result')
    ax.plot(x, u_ref_vals, 'r', label='reference')
    ax.legend()

    ax.set(xlabel='x', ylabel='u',
        title='u(x)')
    ax.grid()

    plt.show()
    
    fig, ax = plt.subplots()
    ax.plot(x, np.absolute(u[n] - u_ref_vals), 'r', label='error')
    ax.legend()

    ax.set(xlabel='x', ylabel='u',
        title='error(x)')
    ax.grid()

    plt.show()

    sigma = 0
    if scheme == 0:
        sigma = 0.1
    elif scheme == 1:
        sigma = 0.6
    else:
        sigma = 0.5

    cur_h = 0.05
    
    hs = []
    taus = []
    
    n_0 = 10
    n_n = 70
    
    for i in range(n_0, n_n):
        cur_h = (x_end - x_begin) / (i - 1)
        cur_tau = sigma * cur_h * cur_h / a
        hs.append(cur_h)
        taus.append(cur_tau)

    hs.reverse()
    taus.reverse()

    errs = []
    for h, tau in zip(hs, taus):
        max_err = 0.0
        u = solver(a, b, c, q, x_begin, x_end, t_begin, t_end, h, tau, phi1, phi2, psi1, psi2, f)

        nx = round((x_end - x_begin) / h) + 1
        nt = round((t_end - t_begin) / tau) + 1

        x = np.linspace(x_begin, x_end, nx)

        for k in range(nt):
            u_ref_vals = u_ref(x, t_begin + tau * k)
            err = np.max(np.absolute(u[k] - u_ref_vals))
            if err > max_err:
                max_err = err
                
        errs.append(max_err)

    errs = np.array(errs)

    log_steps = np.log(hs)
    log_errs = np.log(errs)
    
    fig, ax = plt.subplots()
    
    ax.plot(log_steps, log_errs, 'r', label='max_error')
    ax.legend()

    ax.set(xlabel='h1', ylabel='max_error',
        title='max_error(h)')
    ax.grid()

    plt.show()
    
    approximation_order = (log_errs[-1] - log_errs[0]) / (log_steps[-1] - log_steps[0])
    print(f'{approximation_order=}')

interact_manual(solve, scheme=[('Крест-схема', 0), ('Неявная схема', 1)],
                 approximation=[('Двухточечная первого порядка', 0), ('Двухточечная второго порядка', 1)],
                 a='1', b='2', c='-3', q='2', x_begin='0.0', x_end=str(math.pi), t_begin=t_begin_widget, t_end=t_end_widget, h='0.05', tau='0.0005', current_t=current_t_widget)
print()

interactive(children=(Dropdown(description='scheme', options=(('Крест-схема', 0), ('Неявная схема', 1)), value…